In [1]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import itertools as it
import pickle

In [2]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

typeEchantillon : 
- fixe pour utiliser tailleEchantillon
- variable pour utiliser nombre,increment
- total pour utiliser tout le lexique

In [3]:
filePrefix="MGC-160104"
lexiquePrefix="MGC-160104"
nbEchantillons=6
tailleEchantillon=15000

##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [4]:
debug=False

In [5]:
bdlexiqueIn = unicode(u"èò")
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO")
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [6]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    return result

In [7]:
with open("/Users/gilles/ownCloud/Recherche/Boye/2015-Data/"+lexiquePrefix+'-Tirages2.pkl', 'rb') as input:
    verbes = pickle.load(input)
minEchantillon=5
tirages={}

In [28]:
print dateheure(),
print "nb tirages %d/%d"%(len(verbes[verbes["tirage"]>0]),verbes[verbes["tirage"]>0]["tirage"].sum()),
print "%4.2f"%(float(verbes[verbes["tirage"]>0]["tirage"].sum())/len(verbes[verbes["tirage"]>0]))

1603151921 nb tirages 74988/231789212 3091.02


In [29]:
def tirerFormes(limite):
    taille=len(verbes[verbes["tirage"]>0])
    while taille < limite:
        tirage=np.random.choice(verbes.index,p=verbes["prob"])
        verbes.loc[tirage,"tirage"]+=1
        taille=len(verbes[verbes["tirage"]>0])
        if taille%100==0:
            if not taille in tirages:
                tirages[taille]=verbes[verbes["tirage"]>0]["tirage"].sum()
                print dateheure(),
                print "nb tirages %d/%d"%(len(verbes[verbes["tirage"]>0]),verbes[verbes["tirage"]>0]["tirage"].sum()),
                print "%4.2f"%(float(verbes[verbes["tirage"]>0]["tirage"].sum())/len(verbes[verbes["tirage"]>0]))

In [32]:
%%time
for echantillon in range(minEchantillon,nbEchantillons+1):
    tirerFormes(echantillon*tailleEchantillon)
    extrait=verbes[verbes["tirage"]>0]
    echantillonPrefix="-%02d"%echantillon
    paradigmes=pd.pivot_table(extrait, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()
    paradigmes.to_csv(path_or_buf="/Users/gilles/ownCloud/Recherche/Boye/2015-Data/"+filePrefix+echantillonPrefix+"-paradigmesSuite.csv",encoding="utf8",sep=";")
    extrait.to_csv(path_or_buf="/Users/gilles/ownCloud/Recherche/Boye/2015-Data/"+filePrefix+echantillonPrefix+"-tirageSuite.csv",encoding="utf8",sep=";")

KeyboardInterrupt: 

In [56]:
verbes.sort_values(by="tirage",ascending=False)

,ortho,phono,ext,cs,ms,vs,lexeme,freq,prob,case,tirage
173659,est,E,"t""",V,3S,pi,être,2.574944e+12,8.253758e-02,pi3S,11615871
40,a,a,NaN,V,3S,pi,avoir,7.167730e+11,2.297553e-02,pi3S,3230283
434075,était,EtE,"t""",V,3S,ii,être,6.045640e+11,1.937877e-02,ii3S,2728340
11553,ai,E,NaN,V,1S,pi,avoir,3.094820e+11,9.920176e-03,pi1S,1397319
371437,sont,sô,"t""",V,3P,pi,être,2.162210e+11,6.930776e-03,pi3P,974140
415488,voir,vwar,NaN,V,,inf,voir,2.117650e+11,6.787942e-03,inf,955588
171685,es,E,"z""",V,2S,pi,être,2.105690e+11,6.749606e-03,pi2S,949660
31699,avait,avE,"t""",V,3S,ii,avoir,2.092790e+11,6.708256e-03,ii3S,944926
408810,va,va,NaN,V,3S,pi,aller,2.038560e+11,6.534426e-03,pi3S,918755
408809,va,va,NaN,V,2S,pI,aller,2.038560e+11,6.534426e-03,pI2S,918448


In [70]:
print dateheure()
rep="/Users/gilles/ownCloud/Recherche/Boye/2015-Data/"

1602290742


In [31]:
with open("/Users/gilles/ownCloud/Recherche/Boye/2015-Data/"+filePrefix+'-Tirages2.pkl', 'wb') as output:
   pickle.dump(verbes, output, pickle.HIGHEST_PROTOCOL)

In [33]:
paradigmes.to_csv(path_or_buf="/Users/gilles/ownCloud/Recherche/Boye/2015-Data/"+filePrefix+echantillonPrefix+"-paradigmesSuite.csv",encoding="utf8",sep=";")
extrait.to_csv(path_or_buf="/Users/gilles/ownCloud/Recherche/Boye/2015-Data/"+filePrefix+echantillonPrefix+"-tirageSuite.csv",encoding="utf8",sep=";")